In [1]:
%%configure -f 
{"executorMemory":"200G"}

In [2]:
%%configure -f
{
  "conf": {
    "spark.sql.files.maxPartitionBytes": "1073741824"
  }
}

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import types as T
import pyspark.sql.functions as F
from pyspark.sql.functions import lit
from pyspark.sql.types import IntegerType,BooleanType,DateType,LongType,DoubleType, StringType
from pyspark.sql.functions import row_number,lit,col,when, concat_ws
from pyspark.sql.window import Window
import time
spark = SparkSession.builder \
    .master("yarn") \
    .appName("Wellmark-UHC") \
    .getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1684227770878_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
npi_table = spark.read.parquet("s3://zigna-nsa-data-staging/metadata/Q2-2023/helper-tables/may-wellmark-hosp-ancillary-npi-table/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
npi_table.printSchema()
len(npi_table.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- npi: string (nullable = true)
 |-- entity_type_code: string (nullable = true)
 |-- provider_organization_name_legal_business_name: string (nullable = true)
 |-- provider_credential_text: integer (nullable = true)
 |-- provider_business_practice_location_address_city_name: string (nullable = true)
 |-- provider_business_practice_location_address_state_name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- healthcare_provider_taxonomy_code: string (nullable = true)
 |-- provider_type: string (nullable = true)
 |-- mapped_from: string (nullable = true)
 |-- feb_excellus_npi: string (nullable = true)
 |-- code: string (nullable = true)
 |-- grouping: string (nullable = true)
 |-- classification: string (nullable = true)
 |-- specialization: string (nullable = true)
 |-- display_name: string (nullable = true)
 |-- section: string (nullable = true)
 |-- provider_name: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- state: string (nullable = 

In [6]:
npi_table = npi_table.withColumnRenamed('provider_business_practice_location_address_city_name', 'provider_city_name').withColumnRenamed(
    'provider_business_practice_location_address_state_name', 'provider_state_name').withColumnRenamed('postal_code', 'provider_postal_code').withColumnRenamed('npi', 'master_npi')
# npi_table = npi_table.withColumn("primary_city",npi_table.primary_city.cast(StringType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
npi_table=npi_table.orderBy('provider_name')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
npi_table.groupBy('region').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|              region|count|
+--------------------+-----+
|     682 - DAVENPORT|  155|
|    764 - RAPID CITY|   97|
|  637 - CEDAR RAPIDS|  452|
|        717 - QUINCY|   20|
|679 - DES MOINES-...|  583|
|    624 - SIOUX CITY|  231|
|         687 - MINOT|    1|
|       631 - OTTUMWA|   39|
|         652 - OMAHA|  135|
|   725 - SIOUX FALLS|  373|
|     611 - ROCHESTER|   82|
+--------------------+-----+

In [9]:
npi_table.distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2168

In [10]:
npi_table.groupBy('provider_type').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----+
|provider_type|count|
+-------------+-----+
|    Ancillary| 1966|
|     Hospital|  202|
+-------------+-----+

In [11]:
npi_table[['master_npi']].distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2168

In [12]:
code_table = spark.read.parquet("s3://zigna-nsa-data-staging/metadata/Q2-2023/helper-tables/apr-code-table/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
code_table.printSchema()
len(code_table.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- reference_billing_code_type: string (nullable = true)
 |-- FLAG: string (nullable = true)
 |-- reference_billing_code: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- mapped_billing_code_type: string (nullable = true)
 |-- mapped_billing_code_old: string (nullable = true)
 |-- mapped_billing_code: string (nullable = true)
 |-- mapped_level1_description: string (nullable = true)
 |-- mapped_level4_description: string (nullable = true)
 |-- mapped_description: string (nullable = true)
 |-- code_mapping_category: string (nullable = true)
 |-- mapped_from: string (nullable = true)

12

In [14]:
code_table.groupBy('reference_billing_code_type').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------+-----+
|reference_billing_code_type|count|
+---------------------------+-----+
|                     AP-DRG| 1277|
|                     MS-DRG| 1810|
|                        APC|  894|
|                    APR-DRG| 1277|
|                        CPT|10295|
|                      HCPCS| 7221|
|                   CSTM-ALL|    1|
+---------------------------+-----+

In [15]:
code_table.distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

22775

In [306]:
provider_ref = spark.read.parquet('s3://nsa-payers-data-backup-staging/chandan/RP/Unsaved/2023/05/16/tables/bb45f74e-91a3-4cce-8855-3adf63ff40e3')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [307]:
provider_ref.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- reporting_entity_name: string (nullable = true)
 |-- reporting_entity_type: string (nullable = true)
 |-- billing_code: string (nullable = true)
 |-- billing_code_type: string (nullable = true)
 |-- billing_code_type_version: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- billing_class: string (nullable = true)
 |-- billing_code_modifier: string (nullable = true)
 |-- negotiation_arrangement: string (nullable = true)
 |-- negotiated_type: string (nullable = true)
 |-- negotiated_rate: double (nullable = true)
 |-- expiration_date: string (nullable = true)
 |-- service_code: string (nullable = true)
 |-- additional_information: string (nullable = true)
 |-- npi: string (nullable = true)

In [308]:
provider_ref = provider_ref.withColumnRenamed('renamed_service_code','service_code')
# provider_ref = provider_ref.withColumn("service_code", when(provider_ref["service_code"] == " Urgent", "Urgent Care").otherwise(provider_ref["service_code"]))
provider_ref[['service_code']].distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|service_code|
+------------+
|   All Other|
+------------+

In [309]:
provider_ref.printSchema()
len(provider_ref.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- reporting_entity_name: string (nullable = true)
 |-- reporting_entity_type: string (nullable = true)
 |-- billing_code: string (nullable = true)
 |-- billing_code_type: string (nullable = true)
 |-- billing_code_type_version: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- billing_class: string (nullable = true)
 |-- billing_code_modifier: string (nullable = true)
 |-- negotiation_arrangement: string (nullable = true)
 |-- negotiated_type: string (nullable = true)
 |-- negotiated_rate: double (nullable = true)
 |-- expiration_date: string (nullable = true)
 |-- service_code: string (nullable = true)
 |-- additional_information: string (nullable = true)
 |-- npi: string (nullable = true)

16

In [310]:
# provider_ref = provider_ref.withColumn('reporting_entity_name',lit('UMR'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [311]:
provider_ref = provider_ref.withColumnRenamed('renamed_reporting_entity_name','reporting_entity_name')
provider_ref[['reporting_entity_name']].distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+
|reporting_entity_name|
+---------------------+
|       MultiPlan Inc.|
+---------------------+

In [312]:
provider_ref[['reporting_entity_type']].distinct().show() #healthinsuranceissuer --> Health Insurance Insurer

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+
|reporting_entity_type|
+---------------------+
|          Third Party|
+---------------------+

In [313]:
filtered_provider_references = npi_table.join(provider_ref,npi_table.master_npi == provider_ref.npi, 'inner')
filtered_provider_references.printSchema()
len(filtered_provider_references.columns)
filtered_provider_references.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- master_npi: string (nullable = true)
 |-- entity_type_code: string (nullable = true)
 |-- provider_organization_name_legal_business_name: string (nullable = true)
 |-- provider_credential_text: integer (nullable = true)
 |-- provider_city_name: string (nullable = true)
 |-- provider_state_name: string (nullable = true)
 |-- provider_postal_code: string (nullable = true)
 |-- healthcare_provider_taxonomy_code: string (nullable = true)
 |-- provider_type: string (nullable = true)
 |-- mapped_from: string (nullable = true)
 |-- feb_excellus_npi: string (nullable = true)
 |-- code: string (nullable = true)
 |-- grouping: string (nullable = true)
 |-- classification: string (nullable = true)
 |-- specialization: string (nullable = true)
 |-- display_name: string (nullable = true)
 |-- section: string (nullable = true)
 |-- provider_name: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- state: string (nullable = true)
 |-- county: string (nullable = true)
 |-- regio

In [314]:
req_prov_ref_in_network = filtered_provider_references

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [315]:
code_joined_table = req_prov_ref_in_network.join(code_table, (req_prov_ref_in_network.billing_code == code_table.reference_billing_code) & (req_prov_ref_in_network.billing_code_type == code_table.reference_billing_code_type),"inner")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [316]:
code_joined_table.printSchema()
j2 = code_joined_table
len(code_joined_table.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- master_npi: string (nullable = true)
 |-- entity_type_code: string (nullable = true)
 |-- provider_organization_name_legal_business_name: string (nullable = true)
 |-- provider_credential_text: integer (nullable = true)
 |-- provider_city_name: string (nullable = true)
 |-- provider_state_name: string (nullable = true)
 |-- provider_postal_code: string (nullable = true)
 |-- healthcare_provider_taxonomy_code: string (nullable = true)
 |-- provider_type: string (nullable = true)
 |-- mapped_from: string (nullable = true)
 |-- feb_excellus_npi: string (nullable = true)
 |-- code: string (nullable = true)
 |-- grouping: string (nullable = true)
 |-- classification: string (nullable = true)
 |-- specialization: string (nullable = true)
 |-- display_name: string (nullable = true)
 |-- section: string (nullable = true)
 |-- provider_name: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- state: string (nullable = true)
 |-- county: string (nullable = true)
 |-- regio

In [317]:
j2[["provider_type"]].distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|provider_type|
+-------------+
|    Ancillary|
|     Hospital|
+-------------+

In [318]:
j2[["mapped_level1_description"]].distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+
|mapped_level1_description|
+-------------------------+
|     MDC 03 EAR, NOSE,...|
|     MDC 17 LYMPHATIC,...|
|     MDC 10 ENDOCRINE,...|
|     MDC 01 DISEASES A...|
|     MULTIPLE SIGNIFIC...|
|     MDC 06 DISEASES A...|
|     MDC 16 DISEASES A...|
|     MDC 21 POISONINGS...|
|     MDC 04 DISEASES A...|
|     MDC 13 DISEASES A...|
|     MDC 11 DISEASES A...|
|     MDC 08 DISEASES A...|
|     MDC 23 REHABILITA...|
|     MDC 07 DISEASES A...|
|     MDC 09 DISEASES A...|
|     MDC 19 MENTAL DIS...|
|     MDC 18 INFECTIOUS...|
|     MDC 05 DISEASES A...|
|     MDC 14 PREGNANCY,...|
+-------------------------+

In [319]:
j2 = j2.withColumn("partition_column", \
              when(j2["provider_type"] == "Hospital", j2['provider_name'])\
              .when(j2["provider_type"] == "Ancillary", j2['provider_name'])\
              .otherwise(concat_ws('~', j2.region, j2.mapped_level1_description)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [320]:
j2.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['master_npi', 'entity_type_code', 'provider_organization_name_legal_business_name', 'provider_credential_text', 'provider_city_name', 'provider_state_name', 'provider_postal_code', 'healthcare_provider_taxonomy_code', 'provider_type', 'mapped_from', 'feb_excellus_npi', 'code', 'grouping', 'classification', 'specialization', 'display_name', 'section', 'provider_name', 'zip', 'state', 'county', 'region', 'provider_credential', 'provider_specialty_description', 'provider_specialty_group', 'inclusion', 'primary_city', 'reporting_entity_name', 'reporting_entity_type', 'billing_code', 'billing_code_type', 'billing_code_type_version', 'name', 'description', 'billing_class', 'billing_code_modifier', 'negotiation_arrangement', 'negotiated_type', 'negotiated_rate', 'expiration_date', 'service_code', 'additional_information', 'npi', 'reference_billing_code_type', 'FLAG', 'reference_billing_code', 'ID', 'mapped_billing_code_type', 'mapped_billing_code_old', 'mapped_billing_code', 'mapped_level1_d

In [321]:
j3 = j2.select('reference_billing_code_type', 'reference_billing_code', 'ID', 'mapped_billing_code_type', 'mapped_billing_code','mapped_billing_code_old', 'mapped_level1_description', 'mapped_level4_description', 'mapped_description','code_mapping_category' ,'partition_column','reporting_entity_name', 'reporting_entity_type','provider_name','provider_city_name','provider_postal_code','provider_state_name','county', 'region', 'zip', 'state', 'primary_city', 'code', 'grouping', 'classification', 'display_name', 'provider_type','npi', 'billing_code', 'billing_code_type', 'billing_code_type_version', 'name', 'description', 'billing_class', 'billing_code_modifier', 'negotiation_arrangement', 'negotiated_type', 'negotiated_rate', 'service_code','provider_credential','provider_specialty_group','provider_specialty_description','additional_information')
j3.printSchema()
len(j3.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- reference_billing_code_type: string (nullable = true)
 |-- reference_billing_code: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- mapped_billing_code_type: string (nullable = true)
 |-- mapped_billing_code: string (nullable = true)
 |-- mapped_billing_code_old: string (nullable = true)
 |-- mapped_level1_description: string (nullable = true)
 |-- mapped_level4_description: string (nullable = true)
 |-- mapped_description: string (nullable = true)
 |-- code_mapping_category: string (nullable = true)
 |-- partition_column: string (nullable = true)
 |-- reporting_entity_name: string (nullable = true)
 |-- reporting_entity_type: string (nullable = true)
 |-- provider_name: string (nullable = true)
 |-- provider_city_name: string (nullable = true)
 |-- provider_postal_code: string (nullable = true)
 |-- provider_state_name: string (nullable = true)
 |-- county: string (nullable = true)
 |-- region: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- s

In [322]:
j3 = j3.drop('reference_billing_code_type').drop('reference_billing_code').drop('description').drop('ID').drop('name').drop('tin_type').drop('tin_value').drop('code')
j3 = j3.drop('billing_code_type').withColumnRenamed('code_mapping_category','r_billing_code_type').withColumnRenamed('billing_code','r_billing_code').withColumnRenamed('mapped_billing_code_type','billing_code_type').withColumnRenamed('mapped_billing_code','billing_code').withColumnRenamed('mapped_description','description').withColumnRenamed('mapped_level1_description','level1_description').withColumnRenamed('mapped_level4_description','level4_description')
j3 = j3.withColumn('plan_name',lit('')).withColumn('date',lit('05-01-2023'))

# For community and enhanced 
# j3 = j3.drop('grouping').withColumnRenamed('provider_specialty_group','grouping').drop('classification').withColumnRenamed('provider_specialty_description','classification').drop('display_name').withColumnRenamed('provider_credential','display_name')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [323]:
j4 = j3.select("date","plan_name", 'reporting_entity_name', 'reporting_entity_type', 'provider_name', 'npi', 'billing_class', 'negotiation_arrangement', 'negotiated_type', 'billing_code_type', 'billing_code', 'level1_description', 'level4_description', 'description', 'negotiated_rate', 'billing_code_modifier', 'provider_city_name', 'provider_postal_code', 'provider_state_name', 'county', 'region', 'zip', 'state', 'primary_city', 'grouping', 'classification', 'display_name', 'provider_type', 'r_billing_code', 'r_billing_code_type', 'billing_code_type_version', 'service_code', 'additional_information', 'partition_column')
j4 = j4.filter(j4.negotiated_rate > 0.0)
j4 = j4.filter(~((j4.negotiated_type == 'percentage') & (j4.negotiated_rate > 250.0)))
j4 = j4.withColumn("negotiated_rate",j4.negotiated_rate.cast(DoubleType()))
# j4 = j4.filter(j4.billing_code_type !='DRG')
j4 = j4.withColumn("npi",j4.npi.cast(StringType()))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [324]:
j4.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+---------------------+---------------------+--------------------+----------+-------------+-----------------------+---------------+-----------------+------------+--------------------+--------------------+--------------------+---------------+---------------------+------------------+--------------------+-------------------+----------+-----------------+-----+-----+------------+--------------+--------------------+--------------------+-------------+--------------+-------------------+-------------------------+------------+----------------------+--------------------+
|      date|plan_name|reporting_entity_name|reporting_entity_type|       provider_name|       npi|billing_class|negotiation_arrangement|negotiated_type|billing_code_type|billing_code|  level1_description|  level4_description|         description|negotiated_rate|billing_code_modifier|provider_city_name|provider_postal_code|provider_state_name|    county|           region|  zip|state|primary_city|      grouping

In [325]:
j4.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

231

In [326]:
j4[['billing_code_type']].distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|billing_code_type|
+-----------------+
|              DRG|
+-----------------+

In [327]:
j4[['billing_class']].distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|billing_class|
+-------------+
|institutional|
+-------------+

In [328]:
j4[['npi']].distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

50

In [329]:
j4[['partition_column']].distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

43

In [330]:
j4[['r_billing_code_type']].distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|r_billing_code_type|
+-------------------+
|            Primary|
|           Unmapped|
|           Teritary|
|          Secondary|
+-------------------+

In [331]:
j4[['partition_column']].distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|    partition_column|
+--------------------+
|NORTHWEST IOWA HO...|
|MERCY HEALTH SERV...|
|VETERANS MEMORIAL...|
|GENESIS HEALTH SY...|
|BLACK HILLS SURGI...|
|IOWA SPECIALTY HO...|
|PELLA REGIONAL HE...|
|PRAIRIE LAKES HEA...|
|STATE UNIVERSITY ...|
|MARY GREELEY MEDI...|
|      AVERA ST MARYS|
|ORANGE CITY MUNIC...|
|HURON REGIONAL ME...|
|VIRGINIA GAY HOSP...|
|STEWART MEMORIAL ...|
|   CITY OF BROOKINGS|
|COVENANT MEDICAL ...|
|      AVERA MCKENNAN|
|BROADLAWNS MEDICA...|
|ST ANTHONY REGION...|
+--------------------+
only showing top 20 rows

In [332]:
j4[['grouping']].distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|      grouping|
+--------------+
|Hospital Units|
|     Hospitals|
+--------------+

In [333]:
j4[['negotiated_type']].distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+
|negotiated_type|
+---------------+
|        derived|
+---------------+

In [334]:
b = j4.filter(j4.negotiated_type.isin(['negotiated']))
b[['partition_column', 'level1_description']].distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+
|partition_column|level1_description|
+----------------+------------------+
+----------------+------------------+

In [335]:
j4[['classification']].distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|      classification|
+--------------------+
| Rehabilitation Unit|
|    Special Hospital|
|General Acute Car...|
|    Psychiatric Unit|
|Psychiatric Hospital|
+--------------------+

In [336]:
a = j4.filter(j4.classification == 'General Acute Care Hospital')
a[['provider_type']].distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|provider_type|
+-------------+
|     Hospital|
+-------------+

In [337]:
j4[['display_name']].distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|        display_name|
+--------------------+
|Psychiatric Hospi...|
|Rural Acute Care ...|
|Critical Access H...|
|    Special Hospital|
|Rehabilitation Ho...|
|General Acute Car...|
|Psychiatric Hospital|
+--------------------+

In [338]:
j4[['level1_description']].distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|  level1_description|
+--------------------+
|MDC 03 EAR, NOSE,...|
|MDC 17 LYMPHATIC,...|
|MDC 10 ENDOCRINE,...|
|MDC 01 DISEASES A...|
|MULTIPLE SIGNIFIC...|
|MDC 06 DISEASES A...|
|MDC 16 DISEASES A...|
|MDC 21 POISONINGS...|
|MDC 04 DISEASES A...|
|MDC 13 DISEASES A...|
|MDC 11 DISEASES A...|
|MDC 08 DISEASES A...|
|MDC 23 REHABILITA...|
|MDC 07 DISEASES A...|
|MDC 09 DISEASES A...|
|MDC 19 MENTAL DIS...|
|MDC 18 INFECTIOUS...|
|MDC 05 DISEASES A...|
|MDC 14 PREGNANCY,...|
+--------------------+

In [339]:
j4[['provider_type']].distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|provider_type|
+-------------+
|    Ancillary|
|     Hospital|
+-------------+

In [340]:
ui_table = j4

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [341]:
if str(ui_table.schema["date"].dataType) == 'StringType' and str(ui_table.schema["npi"].dataType) == 'StringType' and str(ui_table.schema["state"].dataType) == 'StringType' and str(ui_table.schema["r_billing_code_type"].dataType) == 'StringType' and str(ui_table.schema["r_billing_code"].dataType) == 'StringType' and str(ui_table.schema["negotiated_rate"].dataType) == 'DoubleType' and str(ui_table.schema["partition_column"].dataType) == 'StringType' and len(ui_table.columns) == 34 and str(ui_table.schema["reporting_entity_name"].dataType) == 'StringType' and str(ui_table.schema["billing_code"].dataType) == 'StringType' and str(ui_table.schema["billing_code_type"].dataType) == 'StringType' and str(ui_table.schema["billing_code_type_version"].dataType) == 'StringType' and str(ui_table.schema["billing_class"].dataType) == 'StringType' and str(ui_table.schema["billing_code_modifier"].dataType) == 'StringType' and str(ui_table.schema["negotiation_arrangement"].dataType) == 'StringType' and str(ui_table.schema["negotiated_type"].dataType) == 'StringType' and str(ui_table.schema["service_code"].dataType) == 'StringType' and str(ui_table.schema["additional_information"].dataType) == 'StringType' and str(ui_table.schema["level1_description"].dataType) == 'StringType' and str(ui_table.schema["description"].dataType) == 'StringType' and str(ui_table.schema["provider_name"].dataType) == 'StringType' and str(ui_table.schema["provider_city_name"].dataType) == 'StringType' and str(ui_table.schema["provider_state_name"].dataType) == 'StringType' and str(ui_table.schema["provider_postal_code"].dataType) == 'StringType' and str(ui_table.schema["county"].dataType) == 'StringType' and str(ui_table.schema["region"].dataType) == 'StringType' and str(ui_table.schema["provider_type"].dataType) == 'StringType' and str(ui_table.schema["grouping"].dataType) == 'StringType' and str(ui_table.schema["classification"].dataType) == 'StringType' and str(ui_table.schema["display_name"].dataType) == 'StringType' and str(ui_table.schema["plan_name"].dataType) == 'StringType' and str(ui_table.schema["reporting_entity_type"].dataType) == 'StringType' and str(ui_table.schema["level4_description"].dataType) == 'StringType' and str(ui_table.schema["zip"].dataType) == 'StringType' and str(ui_table.schema["primary_city"].dataType) == 'StringType':
    j4.write.partitionBy('partition_column').mode("append").parquet("s3://zigna-nsa-payer-data-parquet-staging/Q2-2023/wellmark/ui-table/wellmark/partitioned_parquet/wellmark-hospital-ancillary/may/datazigna_final_data_model_ui_partitioned_v1/")
    print('true')
else:
    print('error')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

true

In [19]:
for i in range(54,64):    
    idx = 's3://zigna-nsa-payer-data-parquet-staging/Q2-2023/excellus/april-uhc/april-uhc-test-in-network/uhc-test9/' + in_net[i]
    fname = idx
    prname = idx.replace('april-uhc-test-in-network','april-uhc-test-provider-references')
    in_network = spark.read.parquet(fname)
    sc = in_network[['service_code']].distinct().collect()
    scl = []
    for i in sc:
        scl.append(i['service_code'])
    if not 'Office' or 'Urgent Care' or 'IP/OP/ED' or 'All Other' in scl :
        in_network = in_network.withColumn("service_code_v1", when(in_network.service_code == "11","Office")
                                        .when(in_network.service_code == "20","Urgent Care")                            
                                        .when(in_network.service_code.isin(['21','22','23']),"IP/OP/ED")
                                        .otherwise("All Other"))
        in_network = in_network.drop('service_code').withColumnRenamed('service_code_v1','service_code')
    in_network = in_network.distinct()
    provider_ref = spark.read.parquet(prname)
    provider_ref = provider_ref.drop('file_name').drop('processed_file_name')
    filtered_provider_references = npi_table.join(provider_ref,npi_table.master_npi == provider_ref.npi, 'inner')
    unique_pgi = filtered_provider_references[['provider_group_id']].distinct().collect()
    unique_provider_group_id = []
    for i in unique_pgi:
        unique_provider_group_id.append(i['provider_group_id']) 
    filtered_bct = in_network.filter(in_network.billing_code_type.isin(['AP-DRG','APR-DRG','APC','CPT','CSTM-ALL','HCPCS','MS-DRG']))
    req_prov_ref_in_network = filtered_bct.filter(filtered_bct.provider_references.isin(unique_provider_group_id))
    code_joined_table = req_prov_ref_in_network.join(code_table, (req_prov_ref_in_network.billing_code == code_table.reference_billing_code) & (req_prov_ref_in_network.billing_code_type == code_table.reference_billing_code_type),"inner")
    j2 = filtered_provider_references.join(code_joined_table,filtered_provider_references.provider_group_id == code_joined_table.provider_references,"inner")
    j2 = j2.withColumn("provider_group_id",j2.provider_group_id.cast(StringType())).withColumn("provider_references",j2.provider_references.cast(StringType()))
    j2 = j2.withColumn("partition_column", \
                  when(j2["provider_type"] == "Hospital", j2['provider_name'])\
                  .when(j2["provider_type"] == "Ancillary", j2['provider_name'])\
                  .otherwise(concat_ws('~', j2.region, j2.mapped_level1_description)))
    j3 = j2.select('file_name','reference_billing_code_type', 'reference_billing_code', 'ID', 'mapped_billing_code_type', 'mapped_billing_code','mapped_billing_code_old', 'mapped_level1_description', 'mapped_level4_description', 'mapped_description','code_mapping_category' ,'partition_column','reporting_entity_name', 'reporting_entity_type','provider_name','provider_city_name','provider_postal_code','provider_state_name','county', 'region', 'zip', 'state', 'primary_city', 'code', 'grouping', 'classification', 'display_name', 'provider_type','npi', 'tin_type', 'tin_value', 'billing_code', 'billing_code_type', 'billing_code_type_version', 'name', 'description', 'billing_class', 'billing_code_modifier', 'negotiation_arrangement', 'negotiated_type', 'negotiated_rate', 'service_code','provider_credential','provider_specialty_group','provider_specialty_description','additional_information')
    j3 = j3.drop('reference_billing_code_type').drop('reference_billing_code').drop('description').drop('ID').drop('name').drop('tin_type').drop('tin_value').drop('code')
    j3 = j3.drop('billing_code_type').withColumnRenamed('code_mapping_category','r_billing_code_type').withColumnRenamed('billing_code','r_billing_code').withColumnRenamed('mapped_billing_code_type','billing_code_type').withColumnRenamed('mapped_billing_code','billing_code').withColumnRenamed('mapped_description','description').withColumnRenamed('mapped_level1_description','level1_description').withColumnRenamed('mapped_level4_description','level4_description')
    j3 = j3.withColumn('plan_name',lit('')).withColumn('date',lit('04-01-2023'))
    # For community and enhanced 
    j3 = j3.drop('grouping').withColumnRenamed('provider_specialty_group','grouping').drop('classification').withColumnRenamed('provider_specialty_description','classification').drop('display_name').withColumnRenamed('provider_credential','display_name')
    j4 = j3.select("date","plan_name", 'reporting_entity_name', 'reporting_entity_type', 'provider_name', 'npi', 'billing_class', 'negotiation_arrangement', 'negotiated_type', 'billing_code_type', 'billing_code', 'level1_description', 'level4_description', 'description', 'negotiated_rate', 'billing_code_modifier', 'provider_city_name', 'provider_postal_code', 'provider_state_name', 'county', 'region', 'zip', 'state', 'primary_city', 'grouping', 'classification', 'display_name', 'provider_type', 'r_billing_code', 'r_billing_code_type', 'billing_code_type_version', 'service_code', 'additional_information', 'partition_column')
    j4 = j4.filter(j4.negotiated_rate > 0.0)
    j4 = j4.filter(~((j4.negotiated_type == 'percentage') & (j4.negotiated_rate > 250.0)))
    j4 = j4.filter(j4.billing_code_type != 'DRG') # remove this for hospitals
    j4 = j4.withColumn("negotiated_rate",j4.negotiated_rate.cast(DoubleType()))
    j4 = j4.withColumn("npi",j4.npi.cast(StringType()))
    cnts = j4.count()
    counts.append(cnts)
    if cnts < 300000000:
        for i in ['Community - IA','Community - SD']:
            j5 = j4.filter(j4.provider_type == i) # remove this for hospitals
            ui_table = j5
            if str(ui_table.schema["date"].dataType) == 'StringType' and str(ui_table.schema["npi"].dataType) == 'StringType' and str(ui_table.schema["state"].dataType) == 'StringType' and str(ui_table.schema["r_billing_code_type"].dataType) == 'StringType' and str(ui_table.schema["r_billing_code"].dataType) == 'StringType' and str(ui_table.schema["negotiated_rate"].dataType) == 'DoubleType' and str(ui_table.schema["partition_column"].dataType) == 'StringType' and len(ui_table.columns) == 34 and str(ui_table.schema["reporting_entity_name"].dataType) == 'StringType' and str(ui_table.schema["billing_code"].dataType) == 'StringType' and str(ui_table.schema["billing_code_type"].dataType) == 'StringType' and str(ui_table.schema["billing_code_type_version"].dataType) == 'StringType' and str(ui_table.schema["billing_class"].dataType) == 'StringType' and str(ui_table.schema["billing_code_modifier"].dataType) == 'StringType' and str(ui_table.schema["negotiation_arrangement"].dataType) == 'StringType' and str(ui_table.schema["negotiated_type"].dataType) == 'StringType' and str(ui_table.schema["service_code"].dataType) == 'StringType' and str(ui_table.schema["additional_information"].dataType) == 'StringType' and str(ui_table.schema["level1_description"].dataType) == 'StringType' and str(ui_table.schema["description"].dataType) == 'StringType' and str(ui_table.schema["provider_name"].dataType) == 'StringType' and str(ui_table.schema["provider_city_name"].dataType) == 'StringType' and str(ui_table.schema["provider_state_name"].dataType) == 'StringType' and str(ui_table.schema["provider_postal_code"].dataType) == 'StringType' and str(ui_table.schema["county"].dataType) == 'StringType' and str(ui_table.schema["region"].dataType) == 'StringType' and str(ui_table.schema["provider_type"].dataType) == 'StringType' and str(ui_table.schema["grouping"].dataType) == 'StringType' and str(ui_table.schema["classification"].dataType) == 'StringType' and str(ui_table.schema["display_name"].dataType) == 'StringType' and str(ui_table.schema["plan_name"].dataType) == 'StringType' and str(ui_table.schema["reporting_entity_type"].dataType) == 'StringType' and str(ui_table.schema["level4_description"].dataType) == 'StringType' and str(ui_table.schema["zip"].dataType) == 'StringType' and str(ui_table.schema["primary_city"].dataType) == 'StringType':
                j5.write.mode("append").parquet(f"s3://zigna-nsa-payer-data-parquet-staging/Q2-2023/wellmark/ui-table/wellmark/flat-table/apr-umr/{i}/6/")
                print('true')
            else:
                print('error')
                error.append(i)
    else:
        big_files.append(i)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true

In [20]:
print(error, big_files,counts)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[] [] [172110011, 169935095, 208970595, 172040905, 172008294, 172127421, 172023497, 169950561, 172127419, 172588945]

In [22]:
idx

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://zigna-nsa-payer-data-parquet-staging/Q2-2023/excellus/april-uhc/april-uhc-test-in-network/uhc-test9/2023-04-01_UMR--Inc-_Third-Party-Administrator_UNITEDHEALTHCARE-NEW-BUSINESS-RATE_UNITEDHEALTHCARE-SELECT-PLUS-NETWORK_-MMI_SEL_in-network-rates'

In [23]:
in_net[53]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2023-04-01_UMR--Inc-_Third-Party-Administrator_UNITEDHEALTHCARE-NEW-BUSINESS-RATE_UNITEDHEALTHCARE-SELECT-PLUS-NETWORK_-MMI_SEL_in-network-rates'

In [21]:
a = spark.read.options(recursiveFileLookup = True).parquet('s3://zigna-nsa-payer-data-parquet-staging/Q2-2023/wellmark/ui-table/wellmark/flat-table/apr-umr/Community - IA/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
a.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5750179748

In [24]:
b= a.distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
b.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/site-packages/sparkmagic/livyclientlib/command.py", line 50, in execute
    output = self._get_statement_output(session, statement_id)
  File "/mnt/notebook-env/lib/python3.7/site-packages/sparkmagic/livyclientlib/command.py", line 94, in _get_statement_output
    session.sleep(retries)
  File "/mnt/notebook-env/lib/python3.7/site-packages/sparkmagic/livyclientlib/livysession.py", line 294, in sleep
    sleep(self._policy.seconds_to_sleep(retries))
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/site-packages/sparkmagic/livyclientlib/command.py", line 58, in execute
    session.wait_for_idle()
  File "/mnt/notebook-env/lib/python3.7/site-packages/sparkmagic/livyclientlib/livysession.py", line 290, in wait_for_idle
    sleep(sleep_time)
KeyboardInterrupt

During handling of the above exception, anoth

TypeError: object of type 'NoneType' has no len()